---
title: Automated imports using dhis2-python-client
short_title: Automated imports
---

## NOTE: THIS NOTEBOOK IS OLD, AND WE PROBABLY WILL NOT SUPPORT MONTHLY ERA5 DATA ANYMORE, SINCE IT'S ALL AVERAGED

If you have read through the tutorials up to this point, you should know all the steps needed to: 

- [Get organisation units](../org-units/intro.md)
- [Download data](../getting-data/intro.md)
- [Aggregate data](../aggregation/intro.md)
- [Basic import into DHIS2](../import-data/using-python-client.ipynb)

By putting all these steps together we can have a single script that can be rerun at regular intervals, to make sure that DHIS2 is continuously updated with the latest climate data. We will demonstrate how to do so in this notebook, with a focus on ERA5-Land data. 

## Connect to DHIS2

First, we connect the python-client to the DHIS2 instance we want to import into. In this case we use one of the public access DHIS2 instances that is continuously reset:

In [ ]:
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Client configuration
cfg = ClientSettings(
  base_url="https://play.im.dhis2.org/stable-2-42-3-1",
  username="admin",
  password="district")

client = DHIS2Client(settings=cfg)
info = client.get_system_info()

# Check if everything is working.
# You should see your current DHIS2 version info.
print("▶ Current DHIS2 version:", info["version"])

▶ Current DHIS2 version: 2.42.3.1


## Create DHIS2 data elements

We also need to create the data elements for importing data into. If you haven't already created your data elements manually, you can follow the steps below to create the data element using the `python-dhis2-client`.

First, create the temperature data element: 

In [2]:
data_element = {
    "name": "2m Temperature - Monthly (ERA5)",
    "shortName": "Temperature - Monthly (ERA5)",
    "valueType": "NUMBER",
    "aggregationType": "AVERAGE",
    "domainType": "AGGREGATE"
}
temperature_created = client.create_data_element(data_element)
print(f"Data element creation status: {temperature_created['status']} and UID: {temperature_created['response']['uid']}")

Data element creation status: OK and UID: QaUd7uG0pkV


Next, create the total precipitation data element: 

In [3]:
data_element = {
    "name": "Total precipitation - Monthly (ERA5)",
    "shortName": "Total precipitation - Monthly (ERA5)",
    "valueType": "NUMBER",
    "aggregationType": "SUM",
    "domainType": "AGGREGATE"
}
precipitation_created = client.create_data_element(data_element)
print(f"Data element creation status: {precipitation_created['status']} and UID: {precipitation_created['response']['uid']}")

Data element creation status: OK and UID: fnUWyFEJPd2


Since we plan to import daily data values, we also create and assign our data element to a new dataset for climate variables with `Monthly` period type:

In [5]:
data_set = {
    "name": "Monthly climate data", 
    "shortName": "Monthly climate data",
    "periodType": "Monthly",
    "dataSetElements": [
        {
            "dataElement": {"id": temperature_created['response']['uid']},
        },
        {
            "dataElement": {"id": precipitation_created['response']['uid']}
        },
    ]
}

data_set_created = client.create_data_set(data_set)
print(f"Data set creation status: {data_set_created['status']} and UID: {data_set_created['response']['uid']}")

Data set creation status: OK and UID: UWekHet1YAf


## Get the DHIS2 organisation units

In order to download and aggregate the data to our DHIS2 organisation units, we also use the python-client to get the level 2 organisation units from our DHIS2 instance:

In [7]:
### Get org units GeoJSON from DHIS2
level = 2
org_units_geojson = client.get_org_units_geojson(level=level)

# Convert GeoJSON to geopandas
import geopandas as gpd
import json
org_units = gpd.read_file(json.dumps(org_units_geojson))
org_units

Skipping field groups: unsupported OGR type: 5


,id,code,name,level,parent,parentGraph,geometry
0,O6uvpzGd5pu,OU_264,Bo,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,fdc6uOvgoji,OU_193190,Bombali,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,lc3eMKXaEfw,OU_197385,Bonthe,2,ImspTQPwCqd,ImspTQPwCqd,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,jUb8gELQApl,OU_204856,Kailahun,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,PMa2VCrupOd,OU_211212,Kambia,2,ImspTQPwCqd,ImspTQPwCqd,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,kJq2mPyFEHo,OU_222616,Kenema,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,qhqAxPSTUXp,OU_226213,Koinadugu,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Vth0fbpFcsO,OU_233310,Kono,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,jmIPBj66vD6,OU_246990,Moyamba,2,ImspTQPwCqd,ImspTQPwCqd,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,TEQlaapDQoK,OU_254945,Port Loko,2,ImspTQPwCqd,ImspTQPwCqd,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


## Check when the data was last imported

Since we want to run this script on a regular interval, we want to avoid importing data that has already been imported. We therefore first want to check the last date for which data was imported for the data element we want to import into. This can be done using the convenience function `analytics_latest_period_for_level()` provided by the python-client.

First, let's get the last imported period for temperature:

In [8]:
temperature_id = 'QaUd7uG0pkV'
latest_temperature_response = client.analytics_latest_period_for_level(de_uid=temperature_id, level=level)
latest_temperature_response

{'meta': {'dataElement': 'QaUd7uG0pkV',
  'level': 2,
  'periodType': 'MONTHLY',
  'calendar': 'iso8601',
  'years_checked': 31},
 'existing': None,
 'next': None}

And then the same for precipitation:

In [9]:
precipitation_id = 'fnUWyFEJPd2'
latest_precipitation_response = client.analytics_latest_period_for_level(de_uid=precipitation_id, level=level)
latest_precipitation_response

{'meta': {'dataElement': 'fnUWyFEJPd2',
  'level': 2,
  'periodType': 'MONTHLY',
  'calendar': 'iso8601',
  'years_checked': 31},
 'existing': None,
 'next': None}

## Downloading and importing the data

The code below shows a simple example that loops through a number of months, downloads the data if the data has not already been imported into DHIS2, aggregate the data, and finally imports into DHIS2. 

In [14]:
from dhis2eo import utils
from dhis2eo.data.cds import era5_land
from dhis2eo.integrations.pandas import dataframe_to_dhis2_json
from earthkit import transforms

# define years and months to download
years = list(range(2015, 2025+1))
months = list(range(1, 12+1))

# download the data
print('Downloading data...')
monthly_data = era5_land.monthly.get(years=years, months=months, bbox=org_units.total_bounds)

# temperature
print('')
print('Temperature:')
## aggregate
print('Aggregating...')
agg = transforms.spatial.reduce(monthly_data['t2m'], org_units, mask_dim='id', how='mean')
agg_df = agg.to_dataframe().reset_index()
agg_df['t2m'] -= 273.15
print(agg_df)
## import
print('Importing...')
payload = dataframe_to_dhis2_json(
    df=agg_df,
    org_unit_col='id',
    period_col='valid_time',
    value_col='t2m',
    data_element_id=temperature_id,
)
#res = client.post("/api/dataValueSets", json=payload, params={"dryRun": "true"}) #.post_data_value_set(temp_payload)
res = client.post_data_value_set(payload)
print(f'Import results: {res['response']['importCount']}')

# precipitation
print('')
print('Precipitation:')
## aggregate
print('Aggregating...')
agg = transforms.spatial.reduce(monthly_data['tp'], org_units, mask_dim='id', how='mean')
agg_df = agg.to_dataframe().reset_index()
agg_df['tp'] *= 1000
print(agg_df)
## import
print('Importing...')
payload = dataframe_to_dhis2_json(
    df=agg_df,
    org_unit_col='id',
    period_col='valid_time',
    value_col='tp',
    data_element_id=precipitation_id,
)
#res = client.post("/api/dataValueSets", json=payload, params={"dryRun": "true"}) 
res = client.post_data_value_set(payload)
print(f'Import results: {res['response']['importCount']}')

dhis2eo.data.utils - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\dhis2eo_data_cds_era5_land_monthly_get_c1416fb567.nc

Temperature:
Aggregating...
               id valid_time  number expver        t2m
0     O6uvpzGd5pu 2015-01-01       0   0001  26.251556
1     O6uvpzGd5pu 2015-02-01       0   0001  26.378998
2     O6uvpzGd5pu 2015-03-01       0   0001  27.337402
3     O6uvpzGd5pu 2015-04-01       0   0001  26.650909
4     O6uvpzGd5pu 2015-05-01       0   0001  25.501617
...           ...        ...     ...    ...        ...
1685  at6UHUQatSo 2025-06-01       0   0001  25.937988
1686  at6UHUQatSo 2025-07-01       0   0001  24.726624
1687  at6UHUQatSo 2025-08-01       0   0001  24.437531
1688  at6UHUQatSo 2025-09-01       0   0001  24.818390
1689  at6UHUQatSo 2025-10-01       0   0005  25.724060

[1690 rows x 5 columns]
Importing...
Import results: {'imported': 0, 'updated': 0, 'ignored': 1690, 'deleted': 0}

Precipitation:
Aggregating...
               id valid_time

## Verify that the data was imported

To verify that the data was imported, let's check again the last imported periods. 

Let's look specifically at temperature:

In [13]:
# STILL EXPLORING
client.analytics_latest_period_for_level(de_uid=temperature_id, level=level)
#data_set_id = 'QVIFXY13UDE'
#client.get_data_value_set({
#    'dataSet': data_set_id, 
#    'dataElement': temperature_id,
#    'orgUnit': 'O6uvpzGd5pu',
#    'period': '202509'
#})

{'meta': {'dataElement': 'QaUd7uG0pkV',
  'level': 2,
  'periodType': 'MONTHLY',
  'calendar': 'iso8601',
  'years_checked': 1},
 'existing': {'id': '20251001',
  'startDate': '2025-10-01',
  'endDate': '2025-10-01'},
 'next': {'id': '20251002',
  'startDate': '2025-10-02',
  'endDate': '2025-10-02'}}